In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/tagging/ontonotes5/processed-ontonotes5.json

In [2]:
# !cp /home/husein/translation/processed-train-ontonotes5.json ./processed-ontonotes5.json

In [3]:
import json

with open('processed-ontonotes5.json') as fopen:
    data = json.load(fopen)

In [4]:
from collections import defaultdict

entities = defaultdict(list)
for i in data:
    entities['text'].append(i[0])
    entities['label'].append(i[1])

In [5]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/EVENT/acara-pertandingan.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/EVENT/cuti-malaysia.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malay-Dataset/master/tagging/entities-OntoNotes5/EVENT/karnival-expo.txt

In [6]:
with open('acara-pertandingan.txt') as fopen:
    event = fopen.read().split('\n')
    
with open('cuti-malaysia.txt') as fopen:
    event.extend(fopen.read().split('\n'))
    
with open('karnival-expo.txt') as fopen:
    event.extend(fopen.read().split('\n'))
    
event = [e.strip() for e in event if len(e) > 0]

In [7]:
results = []
i = 0
while i < len(entities['label']):
    r = []
    if entities['label'][i] == 'EVENT':
        while entities['label'][i] == 'EVENT':
            r.append(i)
            i += 1
        results.append(r)
    i += 1
    
len(results)

1041

In [8]:
import math

def generate_index(l, name, texts, labels, length):
    cp, indices = [], []
    b = length - len(l)
    left = math.ceil(b / 2)
    right = b - left
    minus = l[0] - left
    if minus < 0:
        absolute = np.abs(minus)
        right += absolute
        left -= absolute

    for i in range(l[0] - left, l[0]):
        cp.append(texts[i])
        indices.append(labels[i])

    cp.extend(name)
    indices.extend([labels[l[0]] for _ in range(len(name))])
    try:
        for i in range(l[-1] + 1, l[-1] + right + 1):
            cp.append(texts[i])
            indices.append(labels[i])
    except Exception as e:
        print(e)
        pass
    
    return cp, indices

In [9]:
import json
from sklearn.model_selection import train_test_split
import random

In [10]:
train_X, train_Y = [], []
repeat = 15

for t in event:
    for i in range(repeat):
        x, y = generate_index(results[random.randint(0, len(results) - 1)], 
                              t.split(), entities['text'], entities['label'], 50)
        if len(x) != len(y):
            print('len not same')
            continue
        train_X.append(x)
        train_Y.append(y)

In [11]:
len(train_X)

1665

In [12]:
len(train_X[0])

50

In [13]:
with open('augmentation-event-ontonotes5.json', 'w') as fopen:
    json.dump({'train_X': train_X, 'train_Y': train_Y}, fopen)